In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

_num_embeddings = 4 # 100
_embedding_dim = 3 # 50
_commitment_cost = 0.25

inputs = torch.tensor([
    [1.,2.,3.,4.,5.,6.,7.,8.,9.],
    [4.,5.,6.,7.,8.,9.,1.,2.,3.],
    [7.,8.,9.,1.,2.,3.,4.,5.,6.]
])

_embedding = nn.Embedding(_num_embeddings, _embedding_dim)
_embedding.weight.data = torch.tensor([
    [1.,2.,3.],
    [4.,5.,6.],
    [7.,8.,9.],
    [5.,5.,5.]
])


inputs = inputs.contiguous()
input_shape = inputs.shape

flat_input = inputs.view(-1, _embedding_dim)

repeated_input = flat_input.repeat(1, _embedding.weight.size(0)).view(-1, _embedding_dim)
repeated_embedding = _embedding.weight.repeat(flat_input.size(0), 1).view(-1, _embedding_dim)
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
sims = cos(repeated_input, repeated_embedding).reshape(-1, len(_embedding.weight))

# Encoding
encoding_indices = torch.argmax(sims, dim=1).unsqueeze(1)
encodings = torch.zeros(encoding_indices.shape[0], _num_embeddings, device=inputs.device)
encodings.scatter_(1, encoding_indices, 1)

# Quantize and unflatten
quantized = torch.matmul(encodings, _embedding.weight).view(flat_input.shape)

# Loss
e_latent_loss = F.cosine_embedding_loss(quantized.detach(), flat_input, torch.ones(len(quantized)))
q_latent_loss = F.cosine_embedding_loss(quantized, flat_input.detach(), torch.ones(len(quantized)))

loss = q_latent_loss + _commitment_cost * e_latent_loss

quantized = flat_input + (quantized - flat_input).detach()

c:\users\mstek\pycharmprojects\pytorch-vq-vae\venv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\mstek\pycharmprojects\pytorch-vq-vae\venv\lib\site-packages\numpy\.libs\libopenblas.4sp5sua7cbgxueoc35yp2asoicyyeqzz.gfortran-win_amd64.dll
c:\users\mstek\pycharmprojects\pytorch-vq-vae\venv\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


KeyboardInterrupt: 